In [ ]:
import os
os.environ['OMP_NUM_THREADS'] = '10'
from __future__ import division

import itertools

from cycler import cycler
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

import matplotlib
import fssa

from qutip import *

plt.rcParams['axes.labelsize']  = 18
plt.rcParams['axes.titlesize']  = 16
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16
plt.rcParams['legend.fontsize'] = 16

folder_name = '../Data/'

In [ ]:
def parallel_w0(w0):
    
    H    = w0 * J_x
        
    return steadystate(H, c_ops)

In [ ]:
#Finite-size using https://pyfssa.readthedocs.io/en/stable/index.html

N_vec1 = range(2, 20, 2)
N_vec2 = range(20, 1020, 20)

N_vec  = np.concatenate( (N_vec1, N_vec2), axis = 0 )

for N in N_vec:
    
    print("N = ", N)
    
    w0_vec1       = np.linspace(0.10, 0.9  , 50)
    w0_vec2       = np.linspace(0.90, 1.0  , 50)
    w0_vec3       = np.linspace(1.00, 1.2  , 50)

    w0_vec        = np.concatenate( (w0_vec1, w0_vec2, w0_vec3), axis = 0 )

    J_x           = jmat( N/2, 'x')
    J_y           = jmat( N/2, 'y')
    J_z           = jmat( N/2, 'z')
    
    J_minus       = jmat( N/2, '-')

    kappa         = 1
    c_ops         = [np.sqrt( kappa / (N/2) ) * J_minus]
    
    aux           = parfor( parallel_w0, w0_vec )

    expect_Sx_vec = expect(J_x, aux)
    expect_Sy_vec = expect(J_y, aux)
    expect_Sz_vec = expect(J_z, aux)

    np.save(folder_name + "Sx_vec_N="+str(N)     , expect_Sx_vec )
    np.save(folder_name + "Sy_vec_N="+str(N)     , expect_Sy_vec )
    np.save(folder_name + "Sz_vec_N="+str(N)     , expect_Sz_vec )
    
    np.save(folder_name + "inversion_w0_vec_N="+str(N)  , w0_vec )

In [ ]:
N_vec       = range(20, 800, 40)

ls          = N_vec
a           = []

st          = 10

for N in N_vec:    
    aux = np.load(folder_name + "Sz_vec_N="+str(N)+".npy" )
    aux = aux[st:-1]
    a.append( np.divide(-aux, N)  )
    
w0_vec       = np.load(folder_name + "inversion_w0_vec_N="+str(N)+".npy" )

rhos         = w0_vec[st:-1]

da           = np.multiply(a , 0.005)

ret          = fssa.autoscale(ls, rhos, a, da, 0.99, 1.45, -0.45)

print(ret)

auto_scaled_data_1 = fssa.scaledata(ls, rhos, a, da, ret.rho, ret.nu, ret.zeta)

print("wcritical = ", ret.rho, ret.drho)
print("nu = ", ret.nu, ret.dnu)
print("zeta = ", ret.zeta, ret.dzeta)

print(ret.fun)